In [ ]:
import dreadnode as dn
from rich import print

dn.configure(
    server="http://localhost:8000",
    token=""
)

Dreadnode Configuration: (from explicit parameters)

Server: http://localhost

Organization: Main Organization

Workspace: Personal Workspace

Project: Default

In [7]:
unmanaged_dataset_valid = dn.load_dataset("../data/packages.parquet")
unmanaged_dataset_valid

Loading dataset from local path...


Dataset(ds=pyarrow.Table
package_name: string
----
package_name: [["ActiproSoftware.Shared.Wpf","ActiproSoftware.SyntaxEditor.Wpf","K2.Framework.SSO.Base-signature","Apsis.NuGet.Twitterizer","Apsis.NuGet.TNValidate",...,"XImage.OCR.Language.Kannada","XImage.OCR.Language.Latin","XImage.OCR.Language.Malayalam","Citms.Common.Core","XImage.OCR.Language.Maltese"],["XImage.OCR.Language.Macedonian","XImage.OCR.Language.MiddleEnglish","XImage.OCR.Language.Luxembourgish","XImage.OCR.Language.Mongolian","XImage.OCR.Language.Marathi",...,"Watch.The.Equalizer.3.Online.Fullmovie.Free.On.Streamings.At.Home","SolanaWallet","BonsaiDevelopment.Protocols.BARP","Pirater-Facebook-Facilement-En-Deux-Minutes-Hack-2023-VVGQzp","coin-master-free-spin-link-new"]], name='18d405513e9f477c90f661ac92a43e28', description=None, version='0.0.1', license=None, tags=None)

In [8]:
dn.save_dataset(
    unmanaged_dataset_valid,
    cache=False
)

In [ ]:
managed_dataset = dn.load_dataset(f"dn://{unmanaged_dataset_valid.name}")
managed_dataset

Loading dataset from remote storage...


Dataset(ds=pyarrow.Table
package_name: string
----
package_name: [["ActiproSoftware.Shared.Wpf","ActiproSoftware.SyntaxEditor.Wpf","K2.Framework.SSO.Base-signature","Apsis.NuGet.Twitterizer","Apsis.NuGet.TNValidate",...,"E2E.SemVer2StableMetadata.191031.172456.8942963","E2E.SemVer1Stable.191031.172456.5505320","E2E.SymbolsPackage.191031.172456.9724233","E2E.Deprecated.191031.172457.0192992","DependencyTestCM"],["XImage.OCR.Language.Macedonian","XImage.OCR.Language.MiddleEnglish","XImage.OCR.Language.Luxembourgish","XImage.OCR.Language.Mongolian","XImage.OCR.Language.Marathi",...,"Watch.The.Equalizer.3.Online.Fullmovie.Free.On.Streamings.At.Home","SolanaWallet","BonsaiDevelopment.Protocols.BARP","Pirater-Facebook-Facilement-En-Deux-Minutes-Hack-2023-VVGQzp","coin-master-free-spin-link-new"],...,["SPLogging.Data","AppLogger_FF_Test","Tools_Serilog","BlazorTagsInput","Black.Beard.CommandLine",...,"Miicrosoft.ViisualStudio.Web.CodeeGenerators.Mvc","Fennorad.AnthropicClient","OnTube","Hosti